# 6.2. Stored Procedures

An SQL stored procedure is a group of pre-compiled SQL statements (prepared SQL code) that can be reused by simply calling it whenever needed.  
It can be used to perform a wide range of database operations such as inserting, updating, or deleting data, generating reports, and performing complex calculations. Stored procedures are very useful because they allow you to encapsulate (bundle) a set of SQL statements as a single unit and execute them repeatedly with different parameters, making it easy to manage and reuse the code.

Stored procedures can contain a combination of DDL and DML statements, as well as control structures and other programming constructs.

### Syntax:

```sql
DELIMITER //
CREATE PROCEDURE procedure_name (
    IN parameter1 datatype, -- optional
    IN parameter2 datatype,
    IN parameter3 datatype
    )
BEGIN 
    --SQL statement to be executed when the SP is called
    --
    --
END //
DELIMITER ;

-- Setting a delimiter in SP, when using MySQL is neccessary because the semicolon (;) is often used as a statement terminator to indicate the end of a SQL statement. However, when defining stored procedures, you may encounter situations where the default statement terminator conflicts with the semicolons used within the body of the procedure. Developers use the `DELIMITER` statement to temporarily change the statement terminator to a different character. This allows the definition of the stored procedure to contain semicolons without prematurely ending the SQL statement.

### Advantages of Stored Procedures

**Improved performance:** Stored procedures are pre-compiled and stored on the server, so they can be executed more quickly than SQL statements that are sent from client applications.  
**Code reuse:** Stored procedures are pre-compiled and stored on the server, so they can be executed more quickly than SQL statements that are sent from client applications.  
**Reduced Network Traffic:** Because stored procedures are executed on the server, only the results are returned to the client, which reduces network traffic and improves application performance.  
**Better Security:** Stored procedures can be used to enforce security rules and prevent unauthorized access to sensitive data. They can also limit the actions that can be performed by users, making it easier to maintain data integrity and consistency.  
**Simplified maintenance:** By storing SQL code in a single location, it becomes easier to maintain and update the code. This makes it easier to fix bugs, add new functionality, and optimize performance.

## Stored Procedures for creating new records

Creating a simple stored procedure(SP) that at the same time creates a new user and customer. The idea behind this SP is simple, a person wants to register in a webage in order to buy goods.

```sql
DELIMITER // 
CREATE PROCEDURE ceate_user_and_customer(
    -- the parameters that we will need to insert in order to create a new user and customer
    IN userEmail VARCHAR(100),
    IN userPassword VARCHAR(255),
    IN firstName VARCHAR(100),
    IN lastName VARCHAR(100),
    IN phone VARCHAR(20),
    IN address TEXT,
    IN country VARCHAR(40)
)
BEGIN
    -- these variables are declared to store the auto-generated IDs for newly created user and customer
    DECLARE newUserId INT;
    DECLARE newCustomerId INT;

    -- insert the new user into Users table, these fields depends on each specific database preferences, now() in values is used to insert timestamp of exact date when the user is created
    INSERT INTO users (email, password, last_login, registration_date)
    VALUES (userEmail, userPassword, NOW(), NOW());

    -- retrieves the newly auto-genereated users ID and assigns it to the variable
    SET newUserId = LAST_INSERT_ID();

    -- creating a new customer with its specific values, depending on database needs
    INSERT INTO customers (first_name, last_name, phone, address, country, user_id)
    VALUES (firstName, lastName, phone, address, country, newUserId);

    -- retrieves the newly generated customer id and assigns it to the variable
    SET newCustomerId = LAST_INSERT_ID();

    -- when I execute the SP, I like to see the newly inserted IDs in a new table, this is a personal preference and might as well be excluded from the SP
    SELECT newUserId AS user_id, newCustomerId AS customer_id;
END;
//
DELIMITER ;

![new_user_photo](images/create_new_user.png)

**New author:**
```sql
DELIMITER //
CREATE PROCEDURE create_new_author (
	IN new_name VARCHAR(255),
    IN new_birth_date DATE,
    IN new_nationality VARCHAR(255)
)
BEGIN
	INSERT INTO authors (author_name, birth_date, nationality)
    VALUES (new_name, new_birth_date, new_nationality);
END //
DELIMITER ;


![new_author_photo](images/new_author_sp.png)

**New book:**
```sql
DELIMITER //
CREATE PROCEDURE create_new_book (
	IN new_title VARCHAR(255),
    IN new_publication_date DATE,
    IN new_ISBN VARCHAR(13),
    IN new_stock_quantity INT,
    IN new_price DECIMAL (10,2)
    )
BEGIN
	INSERT INTO books (title, publication_date, ISBN, stock_quantity, price)
    VALUES (new_title,new_publication_date, new_ISBN, new_stock_quantity, new_price);
END//
DELIMITER ;


![new_book_photo](images/create_new_book_sp.png)

**New review:**
```sql
DELIMITER //
CREATE PROCEDURE create_review(
    IN p_customer_id INT,
    IN p_book_id INT,
    IN p_rating INT,
    IN p_comments TEXT
)
BEGIN
    -- to validate if the inserted number is in the adequate range, if not, an error message is rased and insertion is aborted
    IF p_rating >= 1 AND p_rating <= 5 THEN
        -- setting variable @current_date to current date,  variable allows to store the current date temporarily for further use within the scope of this session
        SET @current_date = CURDATE();
        
        INSERT INTO reviews (customer_id, book_id, rating, comments, review_date)
        VALUES (p_customer_id, p_book_id, p_rating, p_comments, @current_date);
    ELSE
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Invalid rating. Rating must be between 1 and 5.';
    END IF;
END //
DELIMITER ;


![new_review_photo](images/new_review_sp.png)

**New order:**
```sql
DELIMITER //
CREATE PROCEDURE create_order(
    IN in_customer_id INT
)
BEGIN
    DECLARE new_order_id INT;
    
    -- create a new order with the default status as Pending
    INSERT INTO Orders (customer_id, order_date, status)
    VALUES (in_customer_id, CURDATE(), 'Pending');
    
    -- retrieve the newly generated order_id
    SELECT LAST_INSERT_ID() INTO new_order_id;
    
    -- output the newly generated order_id - my personal preference
    SELECT new_order_id AS order_id;
END;
//
DELIMITER ;


![new_order_photo](images/create_order_sp.png)

**New order items:**
```sql
DELIMITER //
CREATE PROCEDURE create_orderitem(
    IN in_pending_order_id INT,
    IN in_book_id INT,
    IN in_quantity INT
)
BEGIN
    DECLARE book_price DECIMAL(10, 2);
    DECLARE item_subtotal DECIMAL(10, 2);

    -- get the price of the book from the Books table
    SELECT price INTO book_price
    FROM Books
    WHERE book_id = in_book_id;

    -- calculate the subtotal for each book
    SET item_subtotal = book_price * in_quantity;

    -- insert the new order item
    INSERT INTO OrderItems (order_id, book_id, quantity, subtotal)
    VALUES (in_pending_order_id, in_book_id, in_quantity, item_subtotal);
END;
//
DELIMITER ;


![orderitem_photo](images/create_orderitem_sp.png)

**New payment:**
```sql
DELIMITER //
CREATE PROCEDURE create_payment (
    IN pending_order_id INT,
    IN payment_method VARCHAR(50),
    IN payment_status VARCHAR(20)
)
BEGIN
    -- Declares a variable amount of type DECIMAL(10,2) to store the total amount for the pending order
    DECLARE amount DECIMAL(10,2);
    -- Calculates the total amount (subtotal) for the pending order using the SUM function, the result is saved in amount variable
    SELECT sum(subtotal) INTO amount
    FROM orderitems
    WHERE order_id = pending_order_id;

    -- Checks if the provided payment_method is one of the allowed values. If not, it raises an exception with a custom error message
    IF payment_method NOT IN ('Credit Card', 'App Wallet', 'PayPal') THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Invalid payment method. Allowed methods: Credit Card, App Wallet, PayPal.';
        RESIGNAL;
    END IF;

    -- Checks if the provided payment_status is one of the allowed values. If not, it raises an exception with a custom error message
    IF payment_status NOT IN ('Refunded', 'Pending', 'Failed', 'Completed') THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Invalid payment status. Allowed statuses: Refunded, Pending, Failed, Completed.';
        RESIGNAL;
    END IF;

    -- Inserts a new record into the payments table with the current date, payment method, calculated amount, and provided payment status
    INSERT INTO payments (payment_date, payment_method, amount, status)
    VALUES (NOW(), payment_method, amount, payment_status);

    -- Updates the corresponding order in the orders table with the newly inserted payment ID and adjusts the order status based on the payment status
    UPDATE orders
    SET payment_id = LAST_INSERT_ID(),
        status = CASE
            WHEN payment_status = 'Completed' THEN 'Shipped'
            WHEN payment_status = 'Pending' OR payment_status = 'Failed' THEN 'Pending'
            WHEN payment_status = 'Refunded' THEN 'Cancelled'
        END
    WHERE order_id = pending_order_id;
END//
DELIMITER ;


![create_payment_photo](images/create_payment_sp.png)

## Stored Procedures for inserting existing data in the destination table

**Assigning author to a book:**
```sql
DELIMITER //
CREATE PROCEDURE assign_author_to_book(
    IN authorID INT,
    IN bookID INT
)
BEGIN
    -- check if the author and book exist
    IF (EXISTS (SELECT 1 FROM authors WHERE authors_id = authorID) AND
        EXISTS (SELECT 1 FROM books WHERE book_id = bookID)) THEN
        -- insert the association into authorsbooks
        INSERT INTO authorsbooks (authors_id, book_id)
        VALUES (authorID, bookID);
    ELSE
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Author or book does not exist.';
    END IF;
END;
//
DELIMITER ;


**Assigning genre to a book:**
```sql
DELIMITER //
CREATE PROCEDURE assign_genre_to_book(
    IN genreID INT,
    IN bookID INT
)
BEGIN
    -- check if the genre and book exist
    IF (EXISTS (SELECT 1 FROM genres WHERE genre_id = genreID) AND
        EXISTS (SELECT 1 FROM books WHERE book_id = bookID)) THEN
        -- Insert the association into genresbooks
        INSERT INTO genresbooks (book_id, genre_id)
        VALUES (bookID, genreID);
    ELSE
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Genre or book does not exist.';
    END IF;
END;
//
DELIMITER ;


## Stored Procedures with COUNT() function

This kind of stored procedures simplify the process to retrieve different values for data that we have in our database. Instead of writing a query every time we want to check up how many orders, for example, have been delivered, we can just call this procedure and see the result, like this saving tons of time.

Stored procedure to see the count of orders for each possible order status:
```sql
DELIMITER //
CREATE PROCEDURE order_status_count()
BEGIN
	SELECT 
        status, 
        count(order_id) as quantity
    FROM 
        orders
    GROUP BY 1;
END //
DELIMITER ;


The same we can do for Payments table, as there are records with different statuses, and to know how our business processes are going, we need to control and see the statistics for each status:
```sql
DELIMITER //
CREATE PROCEDURE paymentStatusCount ()
BEGIN
	SELECT 
        status, 
        count(payment_id) as quantity
    FROM 
        payments 
    GROUP BY 1;
END //
DELIMITER ;


For example, for data analysis purposes, we can create a procedure that returns us count of payments by the method it has been done, like this we can retrieve information about each payment method and then do appropriate conclusions:
```sql
DELIMITER //
CREATE PROCEDURE paymentMethodCount()
BEGIN
	SELECT 
        payment_method, 
        count(payment_id) as quantity
    FROM 
        payments
    GROUP BY 1;
END //
DELIMITER ;


Procedure to see the total count of payments by each payment method:
```sql
DELIMITER //
CREATE PROCEDURE paymentStatusByMethod()
BEGIN
	SELECT 
        payment_method,
        status,
        count(payment_id) as quantity
    FROM 
        payments
    GROUP BY 1,2;
END //
DELIMITER ;


## Stored Procedures for deleting records from database

As for compliance, auditing and analysis, I have decided to apply soft delete to some of my independent entities in the database.
The trickiest part was to archive customers who no longer are willing to use Bookshop Near Me services. It means not only soft deleting their user and customer information, but to also decide what to do with their corresponding orders and payments. So the idea is to also save payments, order and order items from the deleted customer, and whenever this information is not neccessary, it can be deleted completely by executing corresponding procedure, or by making a trigger which is activated whenever a deletion date is of 90 days of antiquity. 

**Deleting a customer:**
```sql
DELIMITER //
CREATE PROCEDURE archive_customer(
    IN p_customer_id INT
)
BEGIN
    DECLARE v_user_id INT;
    DECLARE v_customer_id INT; 
    DECLARE v_customer_email VARCHAR(100);
    DECLARE v_registration_duration INT;
    DECLARE v_customer_country VARCHAR(40);
    DECLARE v_last_login DATE;

    -- get the associated user_id and customer_id for the customer I`m archiving
    SELECT u.user_id, c.customer_id, u.email, DATEDIFF(NOW(), u.registration_date), c.country, u.last_login
    INTO v_user_id, v_customer_id, v_customer_email, v_registration_duration, v_customer_country, v_last_login
    FROM users u
    INNER JOIN customers c ON u.user_id = c.user_id
    WHERE c.customer_id = p_customer_id;

    -- moving the payments that are associated with the indicated customer to the archived_payments table
    INSERT INTO archived_payments (payment_id, payment_date, payment_method, amount, status, archived_at, archived_by)
    SELECT p.payment_id, p.payment_date, p.payment_method, p.amount, p.status , NOW(), USER()
    FROM payments p
    INNER JOIN orders o ON p.payment_id = o.payment_id
    WHERE o.customer_id = p_customer_id;

    -- when moving proccess has been done, it is save to delete corresponding payment, it is obtained by subquery
    DELETE FROM payments p
	WHERE p.payment_id IN (SELECT payment_id FROM orders WHERE customer_id = p_customer_id);

    -- the same process goes for order items, all the order items for the deleted customer has to be moved to the archive
    INSERT INTO archived_orderitems (order_id, book_id, quantity, subtotal, archived_at, archived_by)
    SELECT oi.order_id, oi.book_id, oi.quantity, oi.subtotal, NOW(), USER()
    FROM orderitems oi
    INNER JOIN orders o ON oi.order_id = o.order_id
    WHERE o.customer_id = p_customer_id;
    
    -- again, when it is done, we can safely delete the records from orderitems table
	DELETE FROM orderitems oi
	WHERE oi.order_id IN (SELECT order_id FROM orders WHERE customer_id = p_customer_id);
    
     -- same process for orders
    INSERT INTO archived_orders (order_id, customer_id, order_date, status, payment_id, archived_at, archived_by)
    SELECT order_id, customer_id, order_date, status, payment_id, NOW(), USER()
    FROM orders
    WHERE customer_id = p_customer_id;

    -- deleting corresponding orders for the indicated customer
	DELETE FROM orders
	WHERE customer_id = p_customer_id;


    -- now, when all customer "belongings" have been taken care of, it is safe to migrate the customer and user information, that we want to see for analytics reasons or compliance, to the deleted_customers table
    INSERT INTO deleted_customers (customer_id, first_name, last_name, phone, address, membership_status, customer_email, registration_duration, customer_country, last_login, deletion_date)
    SELECT c.customer_id, c.first_name, c.last_name, c.phone, c.address, c.membership_status, u.email AS customer_email, DATEDIFF(NOW(), u.registration_date) AS registration_duration, c.country AS customer_country, u.last_login, NOW() AS deletion_date
    FROM customers c
    INNER JOIN users u ON c.user_id = u.user_id
    WHERE c.customer_id = p_customer_id;

    -- after all the moving, it is safe to delete the customer
    DELETE FROM customers
    WHERE customer_id = p_customer_id;

    -- and also corresponding user
    DELETE FROM users
    WHERE user_id = v_user_id;
END //
DELIMITER ;



This procedure is a great example how Stored Procedures can save time, all that I need to insert, when executing this SP, is customer_id and all of the other deletion processes happen automatically... obviously it takes a little time to build such an extended procedure, but, if you feel unsafe, you can always divide this procedure in smaller procedures.

**Deleting an author:**  
My personal preference was not to allow to delete author that still has books assign to it:
```sql
DELIMITER //

CREATE PROCEDURE delete_author(
    IN p_author_id INT
)
BEGIN
    DECLARE v_book_count INT;

    -- check if there are any books by the author in stock
    SELECT COUNT(*)
    INTO v_book_count
    FROM authorsbooks ab
    INNER JOIN books b ON ab.book_id = b.book_id
    WHERE ab.authors_id = p_author_id AND (b.stock_quantity > 0 OR b.stock_quantity IS NULL);

    -- if there are no books in stock, delete the author
    IF v_book_count = 0 THEN
        -- insert the author data into the deleted_authors table
        INSERT INTO deleted_authors (authors_id, author_name, birth_date, nationality, deleted_by, delete_at)
        SELECT authors_id, author_name, birth_date, nationality, USER(), NOW()
        FROM authors
        WHERE authors_id = p_author_id;

        -- delete the author
        DELETE FROM authors WHERE authors_id = p_author_id;
    ELSE
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Cannot delete author. Books by the author are still in stock.';
    END IF;
END //

DELIMITER ;

**Deleting a book:**
```sql
DELIMITER //

CREATE PROCEDURE delete_book(
    IN p_book_id INT
)
BEGIN
    DECLARE v_title VARCHAR(255);
    DECLARE v_publication_date DATE;
    DECLARE v_ISBN VARCHAR(13);
    DECLARE v_stock_quantity INT;
    DECLARE v_avarage_rating DECIMAL(3,1);
    DECLARE v_price DECIMAL(10,2);
    DECLARE v_authors_id INT;
    DECLARE v_author_name VARCHAR(255);

    -- get book data before deletion
    SELECT title, publication_date, ISBN, stock_quantity, avarage_rating, price
    INTO v_title, v_publication_date, v_ISBN, v_stock_quantity, v_avarage_rating, v_price
    FROM books
    WHERE book_id = p_book_id;

    -- get associated author's data, this is a personal preference for easier analysis
    SELECT ab.authors_id, a.author_name
    INTO v_authors_id, v_author_name
    FROM authorsbooks ab
    INNER JOIN authors a ON ab.authors_id = a.authors_id
    WHERE ab.book_id = p_book_id;

    -- insert the deleted book's data into the deleted_books table
    INSERT INTO deleted_books (book_id, title, publication_date, ISBN, stock_quantity, avarage_rating, price, authors_id, author_name, deleted_by, deleted_at)
    VALUES (p_book_id, v_title, v_publication_date, v_ISBN, v_stock_quantity, v_avarage_rating, v_price, v_authors_id, v_author_name, USER(), NOW());

    -- delete the book from authorsbooks, as it appears as well here
    DELETE FROM authorsbooks WHERE book_id = p_book_id;

    -- delete the book
    DELETE FROM books WHERE book_id = p_book_id;
END //

DELIMITER ;


**Deleting loyalty points:**  
Deleting loyalty points exclusively for the deleted customers:
```sql
DELIMITER //
CREATE PROCEDURE delete_loyalty_points()
BEGIN
    -- delete loyalty points for customers in the deleted_customers table
    DELETE lp
    FROM loyaltypoints lp
    INNER JOIN deleted_customers dc ON lp.customer_id = dc.customer_id;
END //
DELIMITER ;


## Stored Procedures for updating

**Restock on books:**  
```sql
DELIMITER //
CREATE PROCEDURE stock_up_books (
    IN bookID INT,
    IN quantity INT
    )
BEGIN
    -- check if the book exists in the low_stock_books view
    IF (EXISTS (SELECT 1 FROM low_stock_books WHERE book_id = bookID)) THEN
        -- if so, increase the stock quantity of the book
        UPDATE books
        SET stock_quantity = stock_quantity + quantity
        WHERE book_id = bookID;
    ELSE
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'The book is not low on stock.';
    END IF;
END;
//
DELIMITER ;


**Update order status:**  
As status a changing attribute, it is neccessary to make it safe to change it
```sql
DELIMITER //
CREATE PROCEDURE update_order_status(
	IN in_order_id INT,
    IN in_new_status VARCHAR(20)
    )
BEGIN
    -- check if the new status is valid
    IF in_new_status IN ('Shipped', 'Delivered', 'Cancelled', 'Pending') THEN
        -- update the status if it's a valid status
        UPDATE orders
        SET status = in_new_status
        WHERE order_id = in_order_id;
    ELSE
        -- raise an error if the status is invalid
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Invalid order status. Order status can only be Shipped, Delivered, Cancelled or Pending';
    END IF;
END;
//
DELIMITER ;


**Update payment status:**  
```sql
DELIMITER //
CREATE PROCEDURE update_payment_status(
    IN paymentID INT,
    IN newPaymentStatus VARCHAR(20)
)
BEGIN
	-- check if newPaymentStatus is a valid value
    IF newPaymentStatus IN ('Pending', 'Completed', 'Failed', 'Refunded') THEN

    -- update the payment status
    UPDATE Payments
    SET status = newPaymentStatus
    WHERE payment_id = paymentID;

    -- update the corresponding order status based on the new payment status
    UPDATE Orders o
    SET o.status = 
        CASE newPaymentStatus
			WHEN 'Pending' THEN 'Shipped'
            WHEN 'Completed' THEN 'Delivered'
            WHEN 'Failed' THEN 'Pending'
            WHEN 'Refunded' THEN 'Cancelled'
            ELSE o.status
        END
    WHERE o.payment_id = paymentID;
    ELSE
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Invalid payment status. Allowed statuses: Pending, Completed, Failed, Refunded.';
    END IF;
END//
DELIMITER ;
